# 本筆記拿鐵達尼號資料集來練習建模。



我們將用Keras來建立一些基本的模型。

* [將檔案存為Pandas DataFrame](#%E5%B0%87%E6%AA%94%E6%A1%88%E5%AD%98%E7%82%BAPandas-DataFrame)
* [空直填補](#%E7%A9%BA%E7%9B%B4%E5%A1%AB%E8%A3%9C)
* [用Keras建立模型：Logistic Regression](#%E7%94%A8Keras%E5%BB%BA%E7%AB%8B%E6%A8%A1%E5%9E%8B%EF%BC%9ALogistic-Regression)
* [用Keras建立模型：Multilayer Perceptron + BatchNorm](#用Keras建立模型：Multilayer-Perceptron-+-BatchNorm)
* [用Keras建立模型：Multilayer Perceptron + Dropout](#%E7%94%A8Keras%E5%BB%BA%E7%AB%8B%E6%A8%A1%E5%9E%8B%EF%BC%9AMultilayer-Perceptron-+-Dropout)

---

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import numpy as np
import re

In [ ]:
# Here's a temporary workout. I replace data in several dataframes, which leads to some warning messages
import warnings
warnings.filterwarnings("ignore")

### 將檔案存為Pandas DataFrame

In [ ]:
data=pd.read_csv("../datasets/titanic/titanic_train.csv") # 輸入資料

In [ ]:
data.info()

[回到頂部](#%E6%9C%AC%E7%AD%86%E8%A8%98%E6%8B%BF%E9%90%B5%E9%81%94%E5%B0%BC%E8%99%9F%E8%B3%87%E6%96%99%E9%9B%86%E4%BE%86%E7%B7%B4%E7%BF%92%E5%BB%BA%E6%A8%A1%E3%80%82)

---

### 空直填補

In [ ]:
cmap=sns.light_palette("navy", reverse=False)
sns.heatmap(data.isnull(),yticklabels=False,cmap=cmap)

In [ ]:
g=sns.factorplot(x="Pclass",y="Age",data=data,kind="box")

In [ ]:
g=sns.factorplot(x="Pclass",y="Age",hue="Sex",data=data,kind="box")

In [ ]:
g=sns.factorplot(x="Sex",y="Age",data=data,kind="box")

In [ ]:
g=sns.factorplot(x="Embarked",y="Age",data=data,kind="box")

決定用Pclass的中位數來填補Age之空值。

In [ ]:
data.groupby("Pclass").median()["Age"]

In [ ]:
def dataClean(df):
    # 資料整理。將空值填補。
    
    nanIndexes={}
    groups=df.groupby("Pclass")
    for name,group in groups:
        nanIndexes[name]=group["Age"][group["Age"].isnull()].index
    data["Age"][nanIndexes[1]]=37
    data["Age"][nanIndexes[2]]=29
    data["Age"][nanIndexes[3]]=24
    
    idxEmbarked=df[df["Embarked"].isnull()].index
    idxDrop=idxEmbarked
    df=df.drop(index=idxDrop)

    df=df.drop(
        columns=["Name","PassengerId","Cabin","Ticket"])

    df["famSize"]=df["SibSp"]+df["Parch"]
    df["Kid"]=df["Age"].apply(lambda x: 1 if x<12 else 0)
    df["Pclass"]=df["Pclass"].astype("object")
    df["Sex"]=df["Sex"].apply(lambda x:0 if "female" in x else 1)
    
    for col in ["Age","Fare"]:
        df[col]=( df[col]-df[col].mean() ) / df[col].std()
    
    df=pd.get_dummies(df)
    
    print("檢查資料整理前是否存在空值:\n",df.isnull().sum(),"\n") # 檢查空值情形。
    return df

def trainTestValSplit(df):
    # 將資料切分為訓練(70%),測試(15%)和驗證(15%)三份。
    train=df.sample(frac=0.7)
    test=df.drop( train.index )
    val=test.sample(frac=0.5)
    test=test.drop( val.index)
    return train,test,val

def dfXYSplit(df,targetName):
    # 將特徵和目標變數切成兩份資料。
    
    dfX=df.drop(columns=targetName)
    dfY=df[targetName]
    
    return dfX,dfY

In [ ]:
data=pd.read_csv("../datasets/titanic/titanic_train.csv") # 輸入資料
data=dataClean(data)
print(data.info() )
data.head(5)

train,test,val=trainTestValSplit(data)

trainX,trainY=dfXYSplit(train,"Survived")
testX,testY=dfXYSplit(test,"Survived")
valX,valY=dfXYSplit(val,"Survived")

資料整理完畢，可以開始建立模型。

[回到頂部](#%E6%9C%AC%E7%AD%86%E8%A8%98%E6%8B%BF%E9%90%B5%E9%81%94%E5%B0%BC%E8%99%9F%E8%B3%87%E6%96%99%E9%9B%86%E4%BE%86%E7%B7%B4%E7%BF%92%E5%BB%BA%E6%A8%A1%E3%80%82)

---

In [ ]:
trainX.shape

In [ ]:
trainY.shape

### 用Keras建立模型：Logistic Regression

In [ ]:
from keras.models import Sequential
from keras.layers import Activation,Dense,Dropout,BatchNormalization
from keras.optimizers import RMSprop,SGD

In [ ]:
model=Sequential()

model.add(Dense(1, input_shape=(trainX.shape[1],),activation='sigmoid') ) 
# initiate RMSprop optimizer
#opt = RMSprop(lr=0.05, decay=1e-6)
opt = SGD(lr=0.05)
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

In [ ]:
history=model.fit(trainX,trainY,epochs=20, validation_data=( valX.values, valY.values ) )

#畫出訓練過程
plt.plot(history.history['acc'],ms=5,marker='o',label='accuracy')
plt.plot(history.history['val_acc'],ms=5,marker='o',label='val accuracy')
plt.legend()
plt.show()

[回到頂部](#%E6%9C%AC%E7%AD%86%E8%A8%98%E6%8B%BF%E9%90%B5%E9%81%94%E5%B0%BC%E8%99%9F%E8%B3%87%E6%96%99%E9%9B%86%E4%BE%86%E7%B7%B4%E7%BF%92%E5%BB%BA%E6%A8%A1%E3%80%82)

---

### 用Keras建立模型：Multilayer Perceptron + BatchNorm

* Reference: 

  BatchNormalization: https://zh-tw.coursera.org/learn/deep-neural-network/lecture/81oTm/why-does-batch-norm-work

  簡單來說，網路較內層的參數若於學習過程中一直改變，會導致較外層的輸出分佈一直產生變化。這件事情將不利於網路的訓練。我們若將這個分佈重新shift, 就可以加速網路訓練。(若看不懂這幾句話，請見以上連結，裡面有比較詳細的說明。)

In [ ]:
batch_size=32

model=Sequential()

model.add(Dense(64, input_shape=(trainX.shape[1],)) )
model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(Dense(64) )
model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(Dense(64) )
model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(Dense(1, input_shape=(trainX.shape[1],),activation='sigmoid') ) 
opt = SGD(lr=0.05)
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

history=model.fit(trainX.values,trainY.values,epochs=20,
                  validation_data=( valX.values, valY.values ), batch_size=batch_size,verbose=1)

#畫出訓練過程
plt.plot(history.history['acc'],ms=5,marker='o',label='accuracy')
plt.plot(history.history['val_acc'],ms=5,marker='o',label='val accuracy')
plt.legend()
plt.show()

[回到頂部](#%E6%9C%AC%E7%AD%86%E8%A8%98%E6%8B%BF%E9%90%B5%E9%81%94%E5%B0%BC%E8%99%9F%E8%B3%87%E6%96%99%E9%9B%86%E4%BE%86%E7%B7%B4%E7%BF%92%E5%BB%BA%E6%A8%A1%E3%80%82)

### 用Keras建立模型：Multilayer Perceptron + Dropout 

添加Dropout layer可減少Overfitting。

* Dropout layer: https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/dropout_layer.html

In [ ]:
batch_size=32

model=Sequential()

model.add(Dense(128, input_shape=(trainX.shape[1],)) )
model.add(Activation("relu"))
model.add(Dropout(0.2))

model.add(Dense(128) )
model.add(Activation("relu"))
model.add(Dropout(0.2))

model.add(Dense(128) )
model.add(Activation("relu"))
model.add(Dropout(0.2))

model.add(Dense(1, input_shape=(trainX.shape[1],),activation='sigmoid') ) 
opt = SGD(lr=0.05)
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

history=model.fit(trainX.values,trainY.values,epochs=20,
                  validation_data=( valX.values, valY.values ), batch_size=batch_size,verbose=1)

#畫出訓練過程
plt.plot(history.history['acc'],ms=5,marker='o',label='accuracy')
plt.plot(history.history['val_acc'],ms=5,marker='o',label='val accuracy')
plt.legend()
plt.show()

[回到頂部](#%E6%9C%AC%E7%AD%86%E8%A8%98%E6%8B%BF%E9%90%B5%E9%81%94%E5%B0%BC%E8%99%9F%E8%B3%87%E6%96%99%E9%9B%86%E4%BE%86%E7%B7%B4%E7%BF%92%E5%BB%BA%E6%A8%A1%E3%80%82)